In [ ]:
import numpy as np
import pandas as pd

: 

In [ ]:
df = pd.read_csv("./Dataset/spam.csv", encoding='latin1')

In [ ]:
df.head()

## DATA CLEANING

In [ ]:
df.info()

In [ ]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [ ]:
df.info()

In [ ]:
df.sample(5)

In [ ]:
df.rename(columns={'v1': 'target', 'v2': 'text'}, inplace=True)

In [ ]:
df.sample(5)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
df['target'] = encoder.fit_transform(df['target']) # changing ham to 0 and spam to 1

In [ ]:
df.sample(5)

In [ ]:
df.isnull().sum()

In [ ]:
# duplicated
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates(keep="first")

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

## 2. EDA

In [ ]:
df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

plt.pie(df['target'].value_counts(), labels=['ham', 'spam'], autopct="%0.2f")
plt.show()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
df['num_chars'] = df['text'].apply(len)

In [ ]:
df.head()

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
df['nums_word']= df['text'].apply(lambda x: len(nltk.word_tokenize(x)))

In [ ]:
df['nums_sent']= df['text'].apply(lambda x: len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df[['num_chars', 'nums_word', 'nums_sent']].describe()

In [ ]:
df[df['target'] == 0][['num_chars','nums_word', 'nums_sent']].describe()

In [ ]:
df[df['target'] == 1][['num_chars','nums_word', 'nums_sent']].describe()

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,5))
sns.histplot(df[df['target'] ==0]['num_chars'])
sns.histplot(df[df['target'] ==1]['num_chars'])

In [ ]:
sns.pairplot(df, hue='target')

In [ ]:
sns.heatmap(df.select_dtypes(include='number').corr(), annot=True)

### DATA PREPROCESSING
- Lower case
- tokenization
- remove special chars
- remove stop words and puntuations
- stemming

In [ ]:
import string
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

In [ ]:
from nltk.stem.porter import PorterStemmer


In [ ]:
def transform_text(text):
    text = text.lower() # lower
    text = nltk.word_tokenize(text) # tokenize
    
    y = []
    for i in text: # remove special chars
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words("english") and i not in string.punctuation:
            y.append(i)
    
    text = y[:]
    y.clear()
    ps = PorterStemmer()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)
    

In [ ]:
transform_text("hello how are you %% ? are you good hassam dancing")

In [ ]:
df['trans_text'] = df['text'].apply(transform_text)

In [ ]:
df.head()

In [ ]:
%pip install wordcloud

from wordcloud import WordCloud


In [ ]:
wc = WordCloud(width=500, height=500, min_font_size=10, background_color='white')

In [ ]:
spam_wc = wc.generate(df[df['target']==1]['trans_text'].str.cat(sep=" "))
plt.imshow(spam_wc)

In [ ]:
ham_wc = wc.generate(df[df['target']==0]['trans_text'].str.cat(sep=" "))
plt.imshow(ham_wc)

In [ ]:
spam_corpus = []
for msg in df[df['target'] == 1]['trans_text'].tolist():
    for word in msg.split():
        spam_corpus.append(word)

spam_corpus

In [ ]:
ham_corpus = []
for msg in df[df['target'] == 0]['trans_text'].tolist():
    for word in msg.split():
        ham_corpus.append(word)

In [ ]:
from collections import Counter

In [ ]:
top_words_df = pd.DataFrame(Counter(spam_corpus).most_common(30), columns=['word', 'count'])
sns.barplot(x='word', y='count', data=top_words_df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
top_words_df_ham = pd.DataFrame(Counter(ham_corpus).most_common(30), columns=['word', 'count'])
sns.barplot(x='word', y='count', data=top_words_df_ham)
plt.xticks(rotation=90)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

: 

In [ ]:
X = cv.fit_transform(df['trans_text']).toarray()